## Colab 환경에서 하는 것을 권장합니다.

# 사전 준비

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/[22-2] DSL session/[0830]TextMining_WordEmbedding/TextMining_WordEmbedding

/content/drive/MyDrive/[22-2] DSL session/[0830]TextMining_WordEmbedding/TextMining_WordEmbedding


In [3]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.co

In [4]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [5]:
# 필요 라이브러리 불러오기
import os, json
from tqdm.notebook import tqdm

from konlpy.tag import Okt

from gensim.models import Word2Vec

import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [6]:
# data 불러오기
data_dir = os.path.join(os.getcwd(), 'data')

corpus = []
for i in tqdm(os.listdir(data_dir)):
    with open(os.path.join(data_dir, i), 'r') as f:
        json_data = json.load(f)
    corpus.append(json_data['sentences'])
corpus = sum(corpus, [])

  0%|          | 0/50 [00:00<?, ?it/s]

데이터를 확인하고 어떻게 전처리를 하고 토큰화를 해야 할 지 고민해보세요!

In [7]:
corpus[:20]

['우리나라 기초연구의 문제가 국가혁신시스템에서 기초연구 전략이 부족하다는 지적이 나왔다',
 '과학기술정책연구원(STEPI)이 3일 한국과학기술회관에서 열린 `기초연구 경쟁력, 질적 혁명에서부터`라는 주제 정책포럼에서 이민형 STEPI 선임연구위원은 “정부가 풀뿌리 지원만 강조하는데 `왜 기초연구를 하는가`에 대한 근본적 질문이 없다”고 비판했다',
 '기초과학이 국가 성장동력 핵심으로 강조되면서 세계 각 국은 기초연구 투자를 확대하고 있다',
 '하지만 우리나라 기초연구 경쟁력은 영향을 미치는 성과가 부족해 과학적 진보, 인류복지 기여 등 국내 경제사회적 문제해결 등 목적을 명시할 필요가 있다는 지적이다',
 '이 위원은 “기초연구 시스템의 미성숙으로 기초연구 축적이 부족하다”면서 “기초연구는 눈에 보이지 않는 문화와 자율이 중요한데 정부가 투자 목표 중심의 정책을 펼치고 있다”고 말했다',
 '그는 세계적 학술지에 내기 위한 연구만 계속 한다는 것은 방향이 잘못됐다고 덧붙였다',
 '연구비 지원 방식도 개편이 필요하다고 주장했다',
 '이 위원은 “대학 교수 개인 지원을 하다 보니 개인주의를 촉발하고 폐쇄성으로 나타난다”며 “지원방식을 개선하지 않고 연구장비 공동화를 아무리 외쳐봐야 소용이 없다',
 '같이 논의할 수 있는 장이 되도록 연구비 지원을 어떤 방식으로 하는지가 변화에 중요한 영향을 미친다”고 말했다',
 '또 인프라를 질적으로 향상시켜야 한다고 강조했다',
 '이 위원은 “연구를 하려면 장비 인프라를 다루는 전문적 엔지니어가 있어서 연구자와 함께 협업을 해야 하는데 이 부분은 지원이 안 된다”면서 “지금은 논문 낸 사람만 칭찬 받는데, 논문 결과는 `부분적`이고 한 나라의 과학 활동의 토대를 닦는 것은 인프라로, 이게 갖춰지지 않으면 노벨상은 기대하기 어렵다”고 전했다',
 '이날 주제 발표 후에는 정광화 전 기초과학지원연구원 원장을 좌장으로 각 분야 전문가 토론이 이어졌다',
 '‘융합’의 바람은 기계공학에서도 거세게 불고 있다',
 '나노

# 1번

자신만의 CustomTokenization을 만들어보세요! 세션에서 말씀 드렸듯이 정답이 있는 것이 아닙니다. 

제가 세션에서 설명했던 것 처럼 pos를 이용하여 품사를 구하고 품사를 통해 stop words를 지정해서 제거해 줄 수 있고, 따로 stopw words를 지정해서 의미 없다고 생각하는 단어들을 제거해도 상관 없습니다!

물론 Okt말고 다른 Tokenizer를 사용해도 괜찮습니다. 

다양한 방법으로 전처리를 진행해보고 왜 그런 전처리와 tokenizing 방법을 사용했는지 설명해주세요!

[여러분의 설명]

In [8]:
okt = Okt()

In [11]:
def CustomTokenization(corpus, stop = ['Josa','Suffix','Punctuation', 'Foreign', 'Number']):
    tokenized = []
    for i, j in okt.pos(corpus, stem = True, norm = True):
        if j in stop:
            continue
        tokenized.append(i)
    return tokenized   

In [12]:
# corpus_tokenized 안에 CustomTokenization의 return값이 들어 있는 2중 list를 만들어 보세요
corpus_tokenized = []
for i in tqdm(corpus):
    corpus_tokenized.append(CustomTokenization(i))

  0%|          | 0/1020 [00:00<?, ?it/s]

In [14]:
corpus_tokenized[:2]

[['우리나라',
  '기초',
  '연구',
  '문제',
  '국가',
  '혁신',
  '시스템',
  '기초',
  '연구',
  '전략',
  '부족하다',
  '지적',
  '나오다'],
 ['과학기술',
  '정책',
  '연구원',
  'STEPI',
  '이',
  '한국',
  '과학기술',
  '회관',
  '열리다',
  '기초',
  '연구',
  '경쟁력',
  '질적',
  '혁명',
  '주제',
  '정책',
  '포럼',
  '이민형',
  'STEPI',
  '선임',
  '연',
  '구',
  '위원',
  '정부',
  '풀',
  '뿌리',
  '지원',
  '강조',
  '하다',
  '왜',
  '기초',
  '연구',
  '하다',
  '대한',
  '근본',
  '질문',
  '없다',
  '고',
  '비판',
  '하다']]

# 2번

이제 word2vec 모델을 학습시켜 봅시다!

1. 사전학습 없이 모델 학습
2. 사전학습 된 모델을 사용하여 학습

두가지를 진행해보세요

hyperparameter는 본인이 하고 싶으신 것으로 하면 됩니다. 단, 사전학습 된 모델을 사용할 때는 size = 200으로 맞추어 주세요!

hyperparameter 설정 기준을 설명해주세요!

[여러분의 설명]

In [22]:
# 사전학습 없이 모델 학습
# 모델 생성
model_no_pretrained = Word2Vec(size = 200, hs = 1, min_count = 0)

# 모델 vocabulary 추가
model_no_pretrained.build_vocab(corpus_tokenized)

total_examples = model_no_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_no_pretrained.wv.vocab))

현재 vocabulary size(V) : 3808


In [26]:
# 모델 학습
model_no_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 20)

(748754, 842100)

In [24]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용
model_no_pretrained.wv.most_similar('바다')

[('수분', 0.9573460817337036),
 ('뭉크', 0.9532921314239502),
 ('감각', 0.9486473798751831),
 ('구글', 0.9444501996040344),
 ('얼굴', 0.9430410861968994),
 ('뚜렷하다', 0.9408591985702515),
 ('벨', 0.9404658079147339),
 ('발명가', 0.9395437240600586),
 ('of', 0.9353799223899841),
 ('Uba', 0.9315657615661621)]

In [19]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용
model_no_pretrained.wv.most_similar('우주')

[('프로톤', 0.57707679271698),
 ('중국', 0.560458779335022),
 ('로켓', 0.5364570617675781),
 ('중단', 0.4999396800994873),
 ('바이코누르', 0.49738192558288574),
 ('회족', 0.4949350655078888),
 ('칼미크인', 0.4926958382129669),
 ('드물다', 0.4853680729866028),
 ('묶다', 0.48401913046836853),
 ('베크', 0.48218125104904175)]

In [29]:
# 사전학습 된 모델을 사용하여 학습
#사전학습 된 모델 불러오기
pretrained = Word2Vec.load('ko.bin')

print('Pretrained model의 단어 수(V) :', len(pretrained.wv.vocab))
print('Pretrained model의 embedding vector의 size(N) :', len(pretrained.wv['강아지']))

# 모델 생성
model_pretrained = Word2Vec(size = 200, hs = 1, min_count = 0)

# 모델 vocabulary 추가
model_pretrained.build_vocab(corpus_tokenized)

total_examples = model_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_pretrained.wv.vocab))

Pretrained model의 단어 수(V) : 30185
Pretrained model의 embedding vector의 size(N) : 200
현재 vocabulary size(V) : 3808


In [30]:
#pretrained model의 학습되어 있는 vocabulary list 불러오기
model_pretrained.build_vocab([list(pretrained.wv.vocab.keys())], update = True)
print('현재 vocabulary size(V) :', len(model_pretrained.wv.vocab))

#embedding vector 까지 불러오기. lockf = 0.0 이면 불러온 벡터에 대해서는 재학습이 불가함. 1.0이면 재학습 가능.
model_pretrained.intersect_word2vec_format('ko.bin.gz', binary = False, lockf = 1.0)

현재 vocabulary size(V) : 31436


In [31]:
# 모델 학습
model_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 20)

(333032, 336840)

In [32]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용
model_pretrained.wv.most_similar('바다')

[('육지', 0.7292351722717285),
 ('호수', 0.6880874633789062),
 ('바닷속', 0.6582979559898376),
 ('늪', 0.6467128992080688),
 ('바닷가', 0.6460920572280884),
 ('급류', 0.6373066902160645),
 ('바닷물', 0.6315383315086365),
 ('절벽', 0.6257826089859009),
 ('바위', 0.6222445964813232),
 ('골짜기', 0.6201075315475464)]

In [33]:
model_pretrained.wv.most_similar('우주')

[('우주선', 0.5852490663528442),
 ('발사체', 0.5602820515632629),
 ('태양계', 0.5472482442855835),
 ('인공위성', 0.5377501845359802),
 ('소유스', 0.5363588333129883),
 ('우주인', 0.5362774133682251),
 ('생명체', 0.5093088150024414),
 ('지구', 0.5077401399612427),
 ('탐사선', 0.5057845115661621),
 ('비행체', 0.4990735948085785)]

# 3번

사전 학습 없이 모델을 학습해도 성능이 괜찮나요?   
사전 학습 없이 학습한 모델과 사전 학습이 된 모델을 사용한 모델 간의 성능 차이가 있었나요? 차이가 있었다면 어떤 부분에서 차이가 나고 왜 그런 차이가 났는지 생각해보고 설명해주세요!

[여러분의 설명]  
- 사전학습 된 모델을 사용한 경우, 해당 모델은 방대한 데이터를 이용한 학습을 통해 파라미터가 최적화 되어 있으므로 성능이 더 좋음을 확인할 수 있었음

# 4번

이제 학습된 모델을 통해 나온 embedding 결과를 2차원, 3차원에 embedding 시켜 봅시다.    
사전학습된 데이터를 사용하면 너무 단어 수가 많아지기 때문에 우리가 처음에 가지고 있던 corpus_tokenized 데이터만 사용하겠습니다.   
PCA, t-SNE를 사용해서 시각화하고 그 두가지 방법의 결과를 비교해보세요!   

추가적으로 무언가 재미있는 시각화 결과 (ex. 이거랑 저거랑 비슷한 위치에 임베딩 됐어요!) 가 있다면 추가적으로 남겨주세요!

[여러분의 설명]

In [34]:
# 시각화할 단어 리스트 생성
unique_text = list(set(sum(corpus_tokenized, [])))
len(unique_text)

3808

In [36]:
# embedding 결과 가져오기
word, vector = [], []
for i in unique_text:
    word.append(i)
    vector.append(model_pretrained.wv[i])

In [45]:
pca = PCA(n_components = 2, random_state = 0).fit_transform(vector)
tsne = TSNE(n_components = 2, random_state = 0, learning_rate = 0.01).fit_transform(vector) # 시각화를 위하여 n_components = 2 로 설정

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.



In [46]:
# 시각화해보세요!

data = go.Scatter(
    x=pca[:,0],
    y=pca[:,1],
    text = ['point #{}'.format(i) for i in range(len(vector))],
    mode='markers',
    marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [42]:
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in range(len(vector))],
    mode='markers',
    marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

# 5번

t-SNE에는 perplexity라는 hyperparameter가 존재했는데요. 5~50 사이의 값에서 robust 한 결과를 보인다고는 하지만 perplexity가 달라지면서 embedding 결과가 미묘하지만 크게 바뀌기도 합니다. perplexity와 n_iter 조합을 달리해보면서 perplexity의 변화가 embedding결과에 어떤 영향을 미치는지 살펴보세요!

Perplexity 가 5 일 때에는 군집이 잘 분리가 되어 임베딩이 되는 것을 확인할 수 있다. 하지만 perplexity 가 커질수록 조금씩 군집 간 경계가 모호해지는 것으로 보아 (5 -> 50), perplexity 가 지나치게 크게 되면 군집간 경계가 사라지는 것을 관찰 가능할 것이다.

In [43]:
# t-SNE 
# 기본 설정: perplexity = 30
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 5, learning_rate = 0.05).fit_transform(vector)
# 시각화
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in range(len(vector))],
    mode='markers',
    marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()


/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [44]:
# t-SNE 
# 기본 설정: perplexity = 30
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 50, learning_rate = 0.05).fit_transform(vector)
# 시각화
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in range(len(vector))],
    mode='markers',
    marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 수고하셨습니다!